In [56]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

path = "C:/Data/chinaUnicom/"
train_data = pd.read_csv(path + "train/train_.csv")
test_data = pd.read_csv(path + "test/test_.csv")

In [50]:
# 特征处理
def increaseFeatures(train):
    # 费用
    train['avg_total_fee']=(train['1_total_fee']+train['2_total_fee']+train['3_total_fee']+train['3_total_fee']).astype(float)/4.0
    train['sum_total_fee'] = (train['1_total_fee']+train['2_total_fee']+train['3_total_fee']+train['3_total_fee']).astype(float)
    train['pay_num_per'] = train['pay_num'].astype(float)/train['pay_times']
    train['contract_timex_feex_over_bill'] = (train['avg_total_fee']*train['contract_time']*train['many_over_bill']).astype(float)
    train['contract_online_time_rate'] = (train['contract_time']/train['online_time']).astype(float)
    # 流量
    train['sum_traffic_month'] = train['month_traffic'] + train['local_trafffic_month']+1
    train['out_local_traffic_rate'] = train['month_traffic'].astype(float)/train['sum_traffic_month']
    train['local_sum_traffic_rate'] = train['local_trafffic_month'].astype(float)/train['sum_traffic_month']
    train['last_month_sum_traffic_rate'] = train['last_month_traffic'].astype(float)/train['sum_traffic_month']
    # 通话时间
    train['sum_caller_receive_time'] = train['local_caller_time']+train['service1_caller_time']+train['service2_caller_time']+1
    train['sum_caller_time'] = train['local_caller_time']+train['service1_caller_time']
    train['caller_receive_time_rate'] = train['sum_caller_time'].astype(float)/(train['sum_caller_receive_time'])
    # 流量+通话时间+费用
    train['month_traffic_total_fee_rate'] = train['sum_traffic_month'].astype(float)/train['sum_caller_receive_time']
    train['month_traffic_caller_rate'] = train['sum_traffic_month'].astype(float)/(train['1_total_fee']+1)
    train['caller_time_total_fee_rate'] = train['sum_caller_receive_time'].astype(float)/(train['1_total_fee']+1)
    #省内、本地流量通话
    train['local_trafficx_local_caller'] = train['local_trafffic_month']*train['local_caller_time'].astype(float)
    train['month_trafficx_service1_call'] = train['month_traffic']*train['service1_caller_time'].astype(float)
    
    return train

train_data = increaseFeatures(train_data)
test_data = increaseFeatures(test_data)

In [51]:
#label 处理函数 转为0--14序列或者One_Hot序列
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def service2label(current_service,One_hot = False):
    le = LabelEncoder()
    tmp = sorted(current_service.unique())
    le.fit(tmp)
    label = le.transform(current_service)
    ohe_tmp = le.transform(tmp)
    if(One_hot==True):
        ohe = OneHotEncoder()
        ohe.fit(ohe_tmp.reshape(-1,1))
        label = ohe.transform(label.reshape(-1,1)).toarray()
    return label
# label 处理
current_service_label = train_data['current_service']
label = service2label(current_service_label,One_hot = False)

In [52]:
train_data.dropna(axis=0,how='any')#删除任何带有NaN值得行
train_data.fillna(0)
train = train_data.drop(['current_service','user_id','sum_total_fee','net_service','complaint_level','former_complaint_num','former_complaint_fee'],axis=1)
original_feature_new = train.columns
allFold_data = train[original_feature_new]

In [53]:
# import pickle
# import lightgbm as lgb
# from sklearn.model_selection import StratifiedKFold,KFold

# k = 1
# stratified_folder = StratifiedKFold(n_splits=10,random_state=0,shuffle=False)

# for iteration, indices in stratified_folder.split(allFold_data,label):
#     X_train = allFold_data.iloc[iteration]
#     y_train = label[iteration]
#     X_validate = allFold_data.iloc[indices]
#     y_validate = label[indices]
#     LGBM_train = lgb.Dataset(X_train, y_train)  
#     LGBM_eval = lgb.Dataset(X_validate, y_validate, reference=LGBM_train)  
#     params = {'num_leaves':28+k,'boosting_type': 'gbdt','objective': 'multiclass','num_class': 15,'metric': 'multi_error','learning_rate': 0.1}
#     model_LGBM = lgb.train(params,LGBM_train,num_boost_round=300,valid_sets=LGBM_eval,early_stopping_rounds=10) 
#     with open( path +'KFold/model_LGBM_'+str(k)+'.pkl','wb') as f:
#             pickle.dump(model_LGBM,f)
#     k += 1

In [54]:
import pickle
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold,KFold

k = 1
stratified_folder = StratifiedKFold(n_splits=10,random_state=0,shuffle=False)

for iteration, indices in stratified_folder.split(allFold_data,label):
    X_train = allFold_data.iloc[iteration]
    y_train = label[iteration]
    X_validate = allFold_data.iloc[indices]
    y_validate = label[indices]
    model_XGB = xgb.XGBClassifier(max_depth=5+ np.random.randint(10), min_child_weight=1, n_estimators=200 +10*np.random.randint(10),learning_rate=0.01+0.05*np.random.randint(10))
    model_XGB.fit(X_train,y_train)
    with open( path +'KFold/model_XGB_'+str(k)+'.pkl','wb') as f:
            pickle.dump(model_XGB,f)
    k += 1


In [55]:
import csv

# for i in range(1,11):
#     model_name = 'model_LGBM_'+str(i)+'.pkl'
#     with open( path +'KFold/'+model_name,'rb') as f:
#             model_LGBM = pickle.load(f) 
#     print("epoch:"+str(i))
#     if i==1:
#         pred = model_LGBM.predict(test_data[original_feature_new],num_iteration=model_LGBM.best_iteration)
#     else:
#         pred += model_LGBM.predict(test_data[original_feature_new],num_iteration=model_LGBM.best_iteration)
for i in range(1,11):
    model_name = 'model_XGB_'+str(i)+'.pkl'
    with open( path +'KFold/'+model_name,'rb') as f:
            model_XGB = pickle.load(f) 
    print("epoch:"+str(i))
    if i==1:
        pred = model_XGB.predict_proba(test_data[original_feature_new])
    else:
        pred += model_XGB.predict_proba(test_data[original_feature_new])
        
pred = np.argmax(pred,axis=1)
labeldic = {}
tmp = sorted(train_data['current_service'].unique())
for i in range(15):
    labeldic[i] = tmp[i]
for j in range(len(pred)):
    pred[j]=labeldic[pred[j]]
    
with open(path+"KFold_submission.csv","w",newline='',encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["user_id","current_service"])
    for i in range(len(test_data)):
        writer.writerow([test_data['user_id'][i],pred[i]])

epoch:1
epoch:2
epoch:3
epoch:4
epoch:5
epoch:6
epoch:7
epoch:8
epoch:9
epoch:10
